[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/it-keeps-disconnecting/blob/main/nightly/waifu_diffusion_v1_4_webui_colab.ipynb)

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q xformers==0.0.20 triton==2.0.0 -U

!git clone -b v2.5 https://dagshub.com/camenduru/ui
!git clone https://huggingface.co/embed/negative /content/ui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/ui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/ui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/ui/scripts/run_n_times.py
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/ui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/ui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/ui-huggingface /content/ui/extensions/stable-diffusion-webui-huggingface
!git clone https://github.com/camenduru/sd-civitai-browser /content/ui/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/ui/extensions/sd-webui-additional-networks
!git clone https://github.com/camenduru/tunnels /content/ui/extensions/tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/ui/extensions/batchlinks-webui
%cd /content/ui
!git reset --hard
!git -C /content/ui/repositories/stable-diffusion-stability-ai reset --hard
!git -C /content/ui/extensions/deforum reset --hard

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/resolve/main/wd-1-4-anime_e1.ckpt -d /content/ui/models/Stable-diffusion -o wd-1-4-anime_e1.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/waifu-diffusion-v1-4/raw/main/wd-1-4-anime_e1.yaml -d /content/ui/models/Stable-diffusion -o wd-1-4-anime_e1.yaml

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/ui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/ui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/ui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/ui/modules/shared.py
!sed -i -e 's/\.sd-webui-controlnet/\.control/g' /content/ui/extensions/deforum/scripts/deforum_helpers/deforum_controlnet.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple